# Geosüsteemide modelleerimine II osa
## Mitmene regressioonanalüüs

__Prof. Evelyn Uuemaa__

Skript võimaldab üles laadida oma CSV-faili ja selle põhjal teha mitmest regressioonanalüüsi.

## Kuidas seda skripti kasutada

Mine hiirega kõige esimesele kastile ja vajuta väikest "Run" nuppu (paremale näitav nooleke) või vajuta Ctrl+Enter, et üksus läbi jooksutada ja minna järgmiste plokkide juurde. Kui midagi valesti läheb, saad sa lihtsasti uuesti algusest alustada. Mõnes plokis on vaja osad sisendväärtused muuta. Need on tähistatud suurte tähtedega MUUDA SIIN

In [1]:
# Impordi vajalikud paketid
from IPython.display import display
import ipywidgets as widgets

import pandas as pd
import seaborn as sns
import statsmodels.api as sm
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

sns.set_theme(style="whitegrid")

%matplotlib inline

Ülejärgmise ploki juures tuleb sul arvutist valida äsja salvestatud CSV-fail. Kui sinu CSV-faili eraldusmärgiks on koma, pole vaja midagi muuta. Kui aga semikoolon, tee muutus järgmises plokis: csv_delimiter = ";".

In [2]:
upload = widgets.FileUpload( accept='.csv', multiple=False )

csv_delimiter = ","
text_encoding = "utf8"

In [ ]:
upload

Kui oled faili oma arvutist valinud, peaks üleslaadimise lahter näitama "Upload (1)", mis tähendab, et fail on nüüd valitud.

Jooksuta läbi järgmine plokk. Tulemuseks peaks olema ilus tabel, mitte ühes jorus tekst. Kui näed väärtuste vahel semikooloneid (;), tuleb teises plokis teha parandus (csv_delimiter = ";").

In [ ]:
filestream_encoding = "utf8"

t = upload.data[0].decode(encoding=filestream_encoding)

with open('tmp.csv', 'w', encoding=text_encoding) as fh:
    fh.write(t)

data = pd.read_csv('tmp.csv', encoding=text_encoding, sep=csv_delimiter)

data.head(10)

### Sõltuva (ennustatav) ja sõltumatu (seletava) tunnuse määratlemine
Defineeri esmalt sõltumatu ja sõltuva tunnuse.

In [ ]:
# Defineeri sõltuv tunnus (Y) ja sõltumatu tunnus (X)
# MUUDA SIIN: vaata oma tabelit ja vali õiged väärtused sõltumatuks ja sõltuvaks tunnuseks
Y = data['Sõltuv tunnus']  # Kirjuta "Sõltuva tunnus" asemele õige veeru nimi tabelist
X = data[['Sõltumatu tunnus 1', 'Sõltumatu tunnus 2']]  # Kirjuta "Sõltumatu tunnus" asemele õige veeru nimi tabelist(kõrgus merepinnast ja x-koordinaat)

## Sisendandmete treening- ja valideerimisandmeteks jagamine
Selleks, et saaksime testida mudelit sõltumatute andmete peal (andmed, mida mudel pole varem näinud), jagame andmed kaheks. 70% andmetest jätame treeningandmeteks, mille alusel loome mudeli ning 30% andmetest jätame valideerimiseks. Valideerimisandmete peal mudelit rakendades saame teada kui hästi mudel tegelikult töötab.

In [19]:
# Tee valim kaheks - treeningandmeteks (70%) ja valideerimisandmeteks (30%)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

## Treeni mudel ja prognoosi


In [ ]:
# Lisa mudelisse konstant
X_train = sm.add_constant(X_train)

# Sobita regressioonimudel
model = sm.OLS(Y_train, X_train).fit()

# Prindi regressiooni tulemused
print("Training Model Summary:")
print(model.summary())

# Prognoosi treening- ja valideerimisandmete peal
Y_train_pred = model.predict(X_train)
Y_test_pred = model.predict(sm.add_constant(X_test))

# Arvuta R² treening- ja valideerimisandmete jaoks
train_r2 = r2_score(Y_train, Y_train_pred)
test_r2 = r2_score(Y_test, Y_test_pred)

print(f"\nTraining R²: {train_r2:.2f}")
print(f"Test R²: {test_r2:.2f}")

# Pane prognoositud väärtused ja tegelikud väärtused graafikule
plt.figure(figsize=(12, 5))

# Treeningandmete graafik
plt.subplot(1, 2, 1)
plt.scatter(Y_train, Y_train_pred, color='blue')
plt.plot([Y_train.min(), Y_train.max()], [Y_train.min(), Y_train.max()], '--', color='red')
plt.xlabel('Mõõdetud väärtused')
plt.ylabel('Prognoositud väärtused')
plt.title('Treeningandmed: Prognoositud vs mõõdetud')
plt.legend()
plt.grid(True)

# Valideerimisandmete graafik
plt.subplot(1, 2, 2)
plt.scatter(Y_test, Y_test_pred, color='green')
plt.plot([Y_test.min(), Y_test.max()], [Y_test.min(), Y_test.max()], '--', color='red')
plt.xlabel('Mõõdetud väärtused')
plt.ylabel('Prognoositud väärtused')
plt.title('Valideerimisandmed: Prognoositud vs mõõdetud')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

## Mudeli tulemuste interpreteerimine
Vaata mudeli treeningandmete (Training) ja valideerimisandmete (Test) determinatsioonikordaja (R2) väärtust. Mõlemate väärtus peaks olema üle 0.7 ja üsna sarnane, mis näitab, et mudel on väga hea - kirjeldab ära suure osa ennustatava muutuja (antud juhul temperatuuri) varieeruvusest ja mudel pole üle sobitatud (valideerimisandmete R2 ei ole oluliselt väiksem kui treeningandmete R2). Lisaks peaks uue mudeli R2 olema tunduvalt parem kui esimese mudeli R2, mis näitab, et uus mudel kirjeldab ära suurema osa temperatuuri varieeruvusest.

## Võta mudeli parameetrid
Viimasena loome regressioonivõrrandi mudeli parameetrite abil. Koostatav võrrand on meie mudel, mille abil saame temperatuuri modelleerida maapinna kõrguste põhjal.

In [ ]:
# Võta mudeli koefitsiendid
coefficients = model.params

# Koosta regressioonivõrrand (mudel)
equation = "Y = " + str(coefficients[0])
for i, col in enumerate(X_train.columns[1:]):  
    equation += f" + ({coefficients[i + 1]}) * {col}"

print("Mudel (regressioonivõrrand):")
print(equation)

Kasuta saadud parameetreid QGISis mudeli arvutusteks.